# asprin-vL Tutorial

***

## 1. Introduction

*asprin-vL* builds on the existing answer set programming (ASP) framework *asprin* and is designed to learn preferences using  labeled data. This tutorial assumes foundational knowledge in ASP syntax and ASP solver/grounder *clingo*. Prior experience with *asprin* would be helpful towards understanding this framework. Tutorial on preference optimization with *asprin* can be found in the *Potassco* user guide, under the chapter 'optimization and preference handling': https://github.com/potassco/guide/releases

Input:
1. pairs of models alongisde a label indicating the preferred model between the pair. The models are made up of binary atoms (either a fact or not).
2. the domain of the binary atoms and the total number of models for training.
3. the types of preference relation allowed and the corresponding range of preference instances as part of a preference statement, in the form of choice rules. 

Output:
1. preference statement(s) in the syntax of *asprin* which best fits the input data 

***

## 2. Example


In [5]:
%%file example_complete.lp

#program domain.
domain(2).                  
atom(food(X)) :- domain(X). 
model(1..2).                

#program examples.
in(food(1),1).              
in(food(1),2).              
in(food(2),2).              
input(1,better,2).          

#program generation.
preference(p1,subset).                                           
{preference(p1,(1,X),1,for(atom(food(X))),())} :- atom(food(X)). 
    
#program preference(subset).
better(P) :- preference(P,subset),not holds(X) : preference(P,_,_,for(X),_), not holds'(X); 
             1 #sum { 1 : not holds(X), holds'(X), preference(P,_,_,for(X),_)}. 
            
#show preference/2.
#show preference/5.

Overwriting example_complete.lp


In [2]:
! python3 asprin_learn.lp example_complete.lp backend.lp -W none

'clingo' is not recognized as an internal or external command,
operable program or batch file.


## 3. Usage

### 3.1 Input
The input consists of 3 necessary program parts and 1 optional program part, each identified with its respective `#program <part>.` directive.

(Please enable the display of line numbers to better follow the explanations below. All explanations refer to the line numbers in the overall example in section 2.  Click on the "view" option and select "Toggle Line Numbers" / "Show Line Numbers" option respectively if you are viewing this in Jupyter Notebook / JupyterLab.
***

#### 3.1.1 Domain
This program part is identified by the directive `#program domain.`

```
#program domain.
domain(2).                  
atom(food(X)) :- domain(X). 
model(1..2).                
```

Line 4 specifys the domain space, which is the range of input allowed for the atom `food` in line 5. Line 6 states the total number of models given to the program.

#### 3.1.2 Example
This program part is identified by the directive `#program example.`

```
#program examples.
in(food(1),1).              
in(food(1),2).              
in(food(2),2).              
input(1,better,2).          
```

Lines 9 to 11 are the input predicates detailing the constitution of each model. The first argument is the atom and the second argument is the model in which the atom is present. In line 9, `in(food(1),1).` means the atom `food(1)` is a fact in model number `1`. Line 12 is the label stating that model `1` is preferred to model `2`. Other preference relations include 'worse', 'eq' for equal, and 'unc' for uncomparable.

#### 3.1.3 Generation
This program part is identified by the directive `#program generation.`

```
#program generation.
preference(p1,subset).                                           
{preference(p1,(1,X),1,for(atom(food(X))),())} :- atom(food(X)). 
```

Line 15 specifies a preference statement p1 and its preference type, in this case the subset type. Line 16 is a choice rule stating the range of preference instances that preference statement p1 can possibly include. The syntax of the predicates `preference/2` and `preference/5`, as well as the formal definition of the subset preference type can be found in the tutorial for *asprin* mentioned in the introduction above.

#### 3.1.4 Library
This program part is identified by the directive `#program preference.` or `#program preference(<identifier>).`

```
#program preference(subset).
better(P) :- preference(P,subset),not holds(X) : preference(P,_,_,for(X),_), not holds'(X); 
             1 #sum { 1 : not holds(X), holds'(X), preference(P,_,_,for(X),_)}.
```

Line 19 is a rule defining the conditions that must be satisfied for the model represented by `holds/1` predicates to be strictly preferred to the model represented by `holds'/1` predicates in terms of the subset relation. This program part is optional as the original *asprin* library containing definitions for various preference types is modified and provided in the repository as 'asprin_vL_lib.lp'

### 3.2 backend.lp

As the name suggests, this file/program part is provided in the repository and requires no actions on the part of the user. It simply has to be included as an input file as shown in section 2. For the detailed description of its content, please refer to the corresponding paper?.

### 3.3 Output
The output is the learned preference statement in asprin format. The format specification of preference statements in *asprin* can be found in the tutorial mentioned in the introduction above.

When the whole learning program is executed, the choice rule in line 16 invokes to include the preference instances (if any) that would best approximate the input labels given the input examples in the solution. In our case here in the tutorial, the following preference instance is part of the solution: 

```
preference(p1,(1,2),1,for(atom(food(2))),()).
```

The preference instance above is then parsed into a *asprin* preference statement as the final output to the program:

```
#preference(p1,subset){
  atom(food(2)) 
}.
```